In [19]:
import pandas as pd
import numpy as np
 
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

import string
import math
import re
from collections import Counter 

#import utilities as ut
import warnings

In [20]:
data = pd.read_json('gg2013.json')
data = pd.DataFrame(data)
data = data['text'].values.tolist()

In [21]:
category_dict = {
    'Best Motion Picture – Drama' :  ['best', 'drama', 'movie', 'motion', 'picture'], 
    'Best Motion Picture – Musical or Comedy' : ['best', 'movie', 'musical', 'comedy', 'motion', 'picture'],
    'Best Motion Picture – Foreign Language' : ['best', 'foreign', 'language', 'motion', 'picture'],
    'Best Motion Picture – Animated' : ['best', 'animated', 'motion', 'picture'],
    'Best Director – Motion Picture' : ['best', 'director', 'motion', 'picture'],
    'Best Actor – Motion Picture Drama' : ['best', 'drama', 'actor', 'motion', 'picture'],
    'Best Actor – Motion Picture Musical or Comedy' : ['best', 'actor', 'musical', 'comedy', 'motion', 'picture'],
    'Best Actress – Motion Picture Drama' : ['best', 'actress', 'drama', 'motion', 'picture'],
    'Best Actress – Motion Picture Musical or Comedy' : ['best', 'musical', 'comedy', 'actress', 'motion', 'picture'],
    'Best Supporting Actor – Motion Picture' : ['best', 'supporting', 'actor', 'motion', 'picture'],
    'Best Supporting Actress – Motion Picture' : ['best', 'supporting', 'actress', 'motion', 'picture'],
    'Best Screenplay – Motion Picture' : ['best', 'screenplay', 'motion', 'picture'],
    'Best Original Score – Motion Picture' : ['best', 'original', 'score', 'motion', 'picture'],
    'Best Original Song – Motion Picture' : ['best', 'original', 'song', 'motion', 'picture'],
    'Cecil B. DeMille Award for Lifetime Achievement in Motion Pictures' : ['cecil', 'demille', 'lifetime', 'achievement'],
    'Best Television Series – Drama' : ['best', 'television', 'tv', 'series', 'drama'],
    'Best Television Series – Musical or Comedy' : ['best', 'television', 'tv', 'series', 'musical', 'comedy'],
    'Best Miniseries or Television Film' : ['best', 'television', 'tv', 'film', 'miniseries'],
    'Best Actor – Television Series Drama' : ['best', 'actor', 'television', 'drama'],
    'Best Actor – Television Series Musical or Comedy' : ['best', 'actor', 'television', 'tv', 'musical', 'comedy'],
    'Best Actor – Miniseries or Television Film' : ['best', 'actor', 'miniseries', 'television', 'film'],
    'Best Actress – Television Series Drama' : ['best', 'actress', 'television', 'tv', 'drama'],
    'Best Actress – Television Series Musical or Comedy' : ['best', 'actress', 'television', 'tv', 'musical', 'comedy'],
    'Best Actress – Miniseries or Television Film' : ['best', 'actress', 'mini', 'miniseries', 'television', 'tv'], 
    'Best Supporting Actor – Series, Miniseries or Television Film' : ['best', 'supporting', 'actor', 'series', 'mini', 'miniseries', 'tv', 'television', 'film'], 
    'Best Supporting Actress – Series, Miniseries or Television Film' : ['best', 'supporting', 'actress', 'series', 'mini', 'miniseries', 'tv', 'television', 'film'], 
    'Carol Burnett Award for Achievement in Television' : ['carol', 'burnett', 'television', 'tv']
}

In [22]:
# return cleaned Tweet as string
# remove stopwords, user handles, punctuation, urls

def cleanTweets(tweet):
    tt = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=True)

    punctuation = list(string.punctuation)
    
    # strip stopwords, punctuation, url components 
    stop = stopwords.words('english') + punctuation + ['t.co', 'http', 'https', '...', '..', ':\\', 'RT', '#']

    strip_nums = re.sub("\d+", "", tweet)
    tokenized = tt.tokenize(strip_nums)
    terms_stop = [term for term in tokenized if term not in stop]
    cleaned = [term for term in terms_stop]
    cleaned = ' '.join(cleaned)
    
    return cleaned

In [ ]:
data['text'] = data['text'].map(cleanTweets)

In [23]:
def filter0(data, list1):
    result = []

    for tweet in data:
        if all(term in tweet.lower() for term in list1):
            result.append(tweet)

    return result

In [24]:
#for all awards and output one for media and one for people

GG_OFFICIAL_AWARDS = ['Best Motion Picture - Drama',
            'Best Motion Picture - Musical or Comedy',
            'Best Director - Motion Picture',
            'Best Original Score - Motion Picture',
            'Best Original Song - Motion Picture',
            'Best Foreign Language Film',
            'Best Animated Feature Film',
            'Best Television Series - Drama',
            'Best Television Series - Comedy Or Musical',
            'Best Screenplay - Motion Picture',
            'Best Performance by an Actor in a Motion Picture - Drama',
            'Best Performance by an Actress in a Motion Picture - Drama',
            'Best Performance by an Actor In A Supporting Role in a Motion Picture',
            'Best Performance by an Actress In A Supporting Role in a Motion Picture',
            'Best Performance by an Actor In A Television Series - Drama',
            'Best Performance by an Actress In A Television Series - Drama',
            'Best Performance by an Actress in a Motion Picture - Comedy Or Musical',
            'Best Performance by an Actor in a Motion Picture - Comedy Or Musical',
            'Best Performance by an Actor In A Television Series - Comedy Or Musical',
            'Best Performance by an Actress In A Television Series - Comedy Or Musical',
            'Best Performance by an Actress in a Supporting Role in a Series',
            'Best Performance by an Actor in a Supporting Role in a Series',
            'Best Mini-Series or Motion Picture made for Television',
            'Best Performance by an Actor in a Mini-Series or Motion Picture Made for Television',
            'Best Performance by an Actress In A Mini-Series or Motion Picture Made for Television']

In [25]:
def separateAwards(award_list):
    possible_people_awards = ['actor',' actress', 'musician', ' singer', 'composer', 'director', 'producer',
                         'screenwriter', 'stage technician', 'author']
    
    people_awards = set()
    media_awards = set()
    
    for category in award_list:
        if any(job in category.lower() for job in possible_people_awards):
            people_awards.add(category)
        else:
            media_awards.add(category)
            
    return people_awards, media_awards

In [27]:
clean_data = []

for tweet in data:
    clean_data.append(cleanTweets(tweet))

In [32]:
def getAwardsAndWinners(tweet_text):
    regex_match = re.search("(?<=Congratulations to ).*(?=[\(\u2026])", tweet_text)
    global test_counter

    if regex_match:
        # get award name and do some standardizing
        award_name = regex_match.group(0)
        award_name = re.sub(r'(television\s+series)|(tv\s+series)', 'TV Series', award_name, flags=re.IGNORECASE)
        
      #  if ut.wordLen(award_name) <= 4:
       #     return
        if "-" not in award_name:
            return
        regex_match = re.search("^[A-Z].*- Best.*", award_name)
        if regex_match:  
            award_name = regex_match.group(0)
            #winner, award= ut.splitWord(award_name)
            #winner = ut.filterBracket(winner)
            #award = ut.filterHash(award)
            
            if winner in res_dict.keys():
                # getReaction doesn't actually get anything for this because of how the tweet are filtered, but that's ok bc we use it later
                getReactions("winners", winner, tweet_text, False) 
                res_dict[winner] = ut.getBetterName(res_dict[winner], award)
            else:
                res_dict[winner] = award
                # getReaction doesn't actually get anything for this because of how the tweet are filtered
                getReactions("winners", winner, tweet_text, True)

In [49]:
if 'rt' in 'robert':
    print("ASODASLDN")

ASODASLDN


In [53]:

answers = {"hosts": ["amy poehler", "tina fey"], 
           "awards": {"best screenplay - motion picture": 
                          {"nominees": ["zero dark thirty", "lincoln", "silver linings playbook", "argo"], 
                           "presenters": ["robert pattinson", "amanda seyfried"], 
                           "winner": "django unchained"}, 
                          "best director - motion picture": 
                          {"nominees": ["kathryn bigelow", "ang lee", "steven spielberg", "quentin tarantino"], 
                           "presenters": ["halle berry"], 
                           "winner": "ben affleck"}}}

In [65]:
nominees = {}
    
year = '2013'
if year is '2013' or year is '2015':
    hardcoded_cat_list = ['cecil b. demille award', 'best motion picture - drama', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best motion picture - comedy or musical', 'best performance by an actress in a motion picture - comedy or musical', 'best performance by an actor in a motion picture - comedy or musical', 'best animated feature film', 'best foreign language film', 'best performance by an actress in a supporting role in a motion picture', 'best performance by an actor in a supporting role in a motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best original score - motion picture', 'best original song - motion picture', 'best television series - drama', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best television series - comedy or musical', 'best performance by an actress in a television series - comedy or musical', 'best performance by an actor in a television series - comedy or musical', 'best mini-series or motion picture made for television', 'best performance by an actress in a mini-series or motion picture made for television', 'best performance by an actor in a mini-series or motion picture made for television', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television']
else:
    hardcoded_cat_list = ['best motion picture - drama', 'best motion picture - musical or comedy', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best performance by an actress in a motion picture - musical or comedy', 'best performance by an actor in a motion picture - musical or comedy', 'best performance by an actress in a supporting role in any motion picture', 'best performance by an actor in a supporting role in any motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best motion picture - animated', 'best motion picture - foreign language', 'best original score - motion picture', 'best original song - motion picture', 'best television series - drama', 'best television series - musical or comedy', 'best television limited series or motion picture made for television', 'best performance by an actress in a limited series or a motion picture made for television', 'best performance by an actor in a limited series or a motion picture made for television', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best performance by an actress in a television series - musical or comedy', 'best performance by an actor in a television series - musical or comedy', 'best performance by an actress in a supporting role in a series, limited series or motion picture made for television', 'best performance by an actor in a supporting role in a series, limited series or motion picture made for television', 'cecil b. demille award']
        
# Make sure our returned dictionary is in the same order as the ones here
for real_cat, our_cat in zip(hardcoded_cat_list, answers['awards'].keys()):
    nominees[real_cat] = (answers['awards'][our_cat]['presenters'])


print(nominees)

{'cecil b. demille award': ['robert pattinson', 'amanda seyfried'], 'best motion picture - drama': ['halle berry']}


In [ ]:
import json

OFFICIAL_AWARDS_1315 = ['cecil b. demille award', 'best motion picture - drama', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best motion picture - comedy or musical', 'best performance by an actress in a motion picture - comedy or musical', 'best performance by an actor in a motion picture - comedy or musical', 'best animated feature film', 'best foreign language film', 'best performance by an actress in a supporting role in a motion picture', 'best performance by an actor in a supporting role in a motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best original score - motion picture', 'best original song - motion picture', 'best television series - drama', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best television series - comedy or musical', 'best performance by an actress in a television series - comedy or musical', 'best performance by an actor in a television series - comedy or musical', 'best mini-series or motion picture made for television', 'best performance by an actress in a mini-series or motion picture made for television', 'best performance by an actor in a mini-series or motion picture made for television', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television']
OFFICIAL_AWARDS_1819 = ['best motion picture - drama', 'best motion picture - musical or comedy', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best performance by an actress in a motion picture - musical or comedy', 'best performance by an actor in a motion picture - musical or comedy', 'best performance by an actress in a supporting role in any motion picture', 'best performance by an actor in a supporting role in any motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best motion picture - animated', 'best motion picture - foreign language', 'best original score - motion picture', 'best original song - motion picture', 'best television series - drama', 'best television series - musical or comedy', 'best television limited series or motion picture made for television', 'best performance by an actress in a limited series or a motion picture made for television', 'best performance by an actor in a limited series or a motion picture made for television', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best performance by an actress in a television series - musical or comedy', 'best performance by an actor in a television series - musical or comedy', 'best performance by an actress in a supporting role in a series, limited series or motion picture made for television', 'best performance by an actor in a supporting role in a series, limited series or motion picture made for television', 'cecil b. demille award']

def get_hosts(year):
    '''Hosts is a list of one or more strings. Do NOT change the name
    of this function or what it returns.'''
    # Your code here
    if year is '2013' or year is '2015':
        with open('1315_results.json') as f:
            data = json.load(f)
    else:
        with open('1819_results.json') as f:
            data = json.load(f)
        
    answers = json.loads(data)
    
    
    hosts = answers['hosts']
                          
    return hosts

def get_awards(year):
    '''Awards is a list of strings. Do NOT change the name
    of this function or what it returns.'''
    # Your code here
    if year is '2013' or year is '2015':
        with open('1315_results.json') as f:
            data = json.load(f)
    else:
        with open('1819_results.json') as f:
            data = json.load(f)
        
    answers = json.loads(data)
    
    
    awards = []
    
    for category in answers['awards'].keys():
        awards.append(category)
    
    return awards

def get_nominees(year):
    '''Nominees is a dictionary with the hard coded award
    names as keys, and each entry a list of strings. Do NOT change
    the name of this function or what it returns.'''
    # Your code here
    if year is '2013' or year is '2015':
        hardcoded_cat_list = OFFICIAL_AWARDS_1315
        with open('1315_results.json') as f:
            data = json.load(f)
    else:
        hardcoded_cat_list = OFFICIAL_AWARDS_1819
        with open('1819_results.json') as f:
            data = json.load(f)
        
    answers = json.loads(data)
    
    
    nominees = {}
        
    # Make sure our returned dictionary is in the same order as the ones here
    for real_cat, our_cat in zip(hardcoded_cat_list, answers['awards'].keys()):
        nominees[real_cat] = (answers['awards'][our_cat]['nominees'])
    
    return nominees

def get_winner(year):
    '''Winners is a dictionary with the hard coded award
    names as keys, and each entry containing a single string.
    Do NOT change the name of this function or what it returns.'''
    # Your code here
    if year is '2013' or year is '2015':
        hardcoded_cat_list = OFFICIAL_AWARDS_1315
        with open('1315_results.json') as f:
            data = json.load(f)
    else:
        hardcoded_cat_list = OFFICIAL_AWARDS_1819
        with open('1819_results.json') as f:
            data = json.load(f)
        
    answers = json.loads(data)
    
    
    winners = {}
        
    # Make sure our returned dictionary is in the same order as the ones here
    for real_cat, our_cat in zip(hardcoded_cat_list, answers['awards'].keys()):
        nominees[real_cat] = (answers['awards'][our_cat]['winner'])
    
    return winners

def get_presenters(year):
    '''Presenters is a dictionary with the hard coded award
    names as keys, and each entry a list of strings. Do NOT change the
    name of this function or what it returns.'''
    # Your code here
    if year is '2013' or year is '2015':
        hardcoded_cat_list = OFFICIAL_AWARDS_1315
        with open('1315_results.json') as f:
            data = json.load(f)
    else:
        hardcoded_cat_list = OFFICIAL_AWARDS_1819
        with open('1819_results.json') as f:
            data = json.load(f)
        
    answers = json.loads(data)
    
    
    presenters = {}
        
    # Make sure our returned dictionary is in the same order as the ones here
    for real_cat, our_cat in zip(hardcoded_cat_list, answers['awards'].keys()):
        nominees[real_cat] = (answers['awards'][our_cat]['presenters'])
    
    return presenters

def pre_ceremony():
    '''This function loads/fetches/processes any data your program
    will use, and stores that data in your DB or in a json, csv, or
    plain text file. It is the first thing the TA will run when grading.
    Do NOT change the name of this function or what it returns.'''
    # Your code here
    # kb and data is a json file
    
    print("Pre-ceremony processing complete.")
    return

def main():
    '''This function calls your program. Typing "python gg_api.py"
    will run this function. Or, in the interpreter, import gg_api
    and then run gg_api.main(). This is the second thing the TA will
    run when grading. Do NOT change the name of this function or
    what it returns.'''
    # Your code here
    with open('twitter_data.json') as f:
        twitter_data = json.load(f)
        
    with open('people_kb.json') as f:
        people_kb = json.load(f)
        
    with open('media_kb.json') as f:
        media_kb = json.load(f)
        
    results1315 = main_exec(OFFICIAL_AWARDS_1315,twitter_data,people_kb,media_kb)
    results1819 = main_exec(OFFICIAL_AWARDS_1819,twitter_data,people_kb,media_kb)
    
    json_results1315 = json.dumps(results1315)
    json_results1819 = json.dumps(results1819)
    
    with open('1315_answers.json', 'w') as f:
        json.dump(json_results1315, f, ensure_ascii=False)
        
    with open('1819_answers.json', 'w') as f:
        json.dump(json_results1819, f, ensure_ascii=False)
    
    return

if __name__ == '__main__':
    main()